[View in Colaboratory](https://colab.research.google.com/github/stanimman/Simple/blob/master/Entity_Embedding.ipynb)

In [1]:
!apt-get install  -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [3]:
!mkdir data
!cd data
!pip install kaggle-cli
!kg download -u 'stanimman' -p 'Legspinner@1' -c 'home-credit-default-risk'

    100% |████████████████████████████████| 81kB 2.8MB/s 
    100% |████████████████████████████████| 5.3MB 4.6MB/s 
    100% |████████████████████████████████| 102kB 20.0MB/s 
    100% |████████████████████████████████| 71kB 20.1MB/s 
  Running setup.py bdist_wheel for kaggle-cli ... - done
  Stored in directory: /content/.cache/pip/wheels/d5/bb/10/c1dd1b08c7433c943cb55c46367ae3f891415e8a37300ff8a7
  Running setup.py bdist_wheel for cliff ... - \ done
  Stored in directory: /content/.cache/pip/wheels/50/00/6d/d4aeb5ccdd47dd76800592b26f943e4959bc705b2d4e6e54e1
  Running setup.py bdist_wheel for configparser ... - done
  Stored in directory: /content/.cache/pip/wheels/a3/61/79/424ef897a2f3b14684a7de5d89e8600b460b89663e6ce9d17c


  Running setup.py bdist_wheel for PrettyTable ... - done
  Stored in directory: /content/.cache/pip/wheels/80/34/1c/3967380d9676d162cb59513bd9dc862d0584e045a162095606
  Running setup.py bdist_wheel for pyperclip ... - done
  Stored in directory: /content/.cache/pip/wheels/58/32/92/2227fba72f0702e4168f084c6dfc2d6d3fd1904634ab61ca6a
Successfully built kaggle-cli cliff configparser PrettyTable pyperclip
downloading https://www.kaggle.com/c/home-credit-default-risk/download/sample_submission.csv.zip

sample_submission.csv.zip 100% |####################| Time: 0:00:00 490.9 KiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/application_test.csv.zip

application_test.csv.zip 100% |#####################| Time: 0:00:00  11.2 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/application_train.csv.zip

application_train.csv.zip 100% |####################| Time: 0:00:01  30.5 MiB/s

downloading https://www.kaggle.com/c/home-credit-defaul

bureau_balance.csv.zip 100% |#######################| Time: 0:00:02  30.0 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/previous_application.csv.zip

previous_application.csv.zip 100% |#################| Time: 0:00:02  28.4 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/credit_card_balance.csv.zip

credit_card_balance.csv.zip 100% |##################| Time: 0:00:03  30.5 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/POS_CASH_balance.csv.zip

POS_CASH_balance.csv.zip 100% |#####################| Time: 0:00:03  31.0 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/installments_payments.csv.zip

installments_payments.csv.zip  64% |##########      | ETA:  0:00:03  28.6 MiB/s

installments_payments.csv.zip 100% |################| Time: 0:00:08  31.3 MiB/s

downloading https://www.kaggle.com/c/home-credit-default-risk/download/HomeCredit_columns_description.csv

HomeCredit_columns_description.csv 100% |###########| Time: 0:00:00 187.7 KiB/s



In [4]:
! apt install unzip


Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 12 not upgraded.
Archive:  application_train.csv.zip
  inflating: application_train.csv   
Archive:  application_test.csv.zip
  inflating: application_test.csv    


In [0]:
# numpy and pandas for data manipulation
import numpy as np
import pandas as pd 

# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# File system manangement
import os

In [9]:
!unzip application_train.csv.zip
!unzip application_test.csv.zip

Archive:  application_train.csv.zip
replace application_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


UnboundLocalError: ignored

In [10]:
!pwd

/content


In [0]:
app_train = pd.read_csv('/content/application_train.csv')
app_test = pd.read_csv('/content/application_test.csv')

In [16]:
app_train.head(2)

,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,...,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,0,0,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,...,0,0,0,0,0,0,1,0,1,0
1,100003,0,0,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,...,0,1,0,0,0,0,0,0,1,0


In [47]:
# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in app_train:
    if app_train[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(app_train[col].unique())) <= 2:
            # Train on the training data
            le.fit(app_train[col])
            # Transform both training and testing data
            app_train[col] = le.transform(app_train[col])
            app_test[col] = le.transform(app_test[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

3 columns were label encoded.


In [48]:
# one-hot encoding of categorical variables
app_train = pd.get_dummies(app_train)
app_test = pd.get_dummies(app_test)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 243)
Testing Features shape:  (48744, 239)


In [49]:
train_labels = app_train['TARGET']

# Align the training and testing data, keep only columns present in both dataframes
app_train, app_test = app_train.align(app_test, join = 'inner', axis = 1)

print('Training Features shape: ', app_train.shape)
print('Testing Features shape: ', app_test.shape)

Training Features shape:  (307511, 239)
Testing Features shape:  (48744, 239)


In [50]:
from sklearn.preprocessing import MinMaxScaler, Imputer

# Drop the target from the training data
if 'TARGET' in app_train:
    train = app_train.drop(columns = ['TARGET'])
else:
    train = app_train.copy()
features = list(train.columns)

# Copy of the testing data
test = app_test.copy()

# Median imputation of missing values
imputer = Imputer(strategy = 'median')

# Scale each feature to 0-1
scaler = MinMaxScaler(feature_range = (0, 1))

# Fit on the training data
imputer.fit(train)

# Transform both training and testing data
train = imputer.transform(train)
test = imputer.transform(app_test)

# Repeat with the scaler
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

print('Training data shape: ', train.shape)
print('Testing data shape: ', test.shape)

Training data shape:  (307511, 239)
Testing data shape:  (48744, 239)


In [51]:
train = train[:246009]
val = train[246009:]
print(train_trail.shape)
print(test_trail.shape)

(246009, 239)
(61502, 239)


In [52]:
train_label = train_labels[:246009] #80% Need to randomize / Highly imbalanced
val_label = train_labels[246009:] 
print(train_label_trail.shape)
print(test_label_trail.shape)

(246009,)
(61502,)


In [0]:
#Split into Train / Validation - Not Now
train=app_train.sample(frac=0.8,random_state=200)
Validation=app_train.drop(train.index)

In [23]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
import torch
print(torch.__version__)
print(torch.cuda.is_available())

tcmalloc: large alloc 1073750016 bytes == 0x5adf8000 @  0x7f959b27e1c4 0x46d6a4 0x5fcbcc 0x4c494d 0x54f3c4 0x553aaf 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54efc1 0x54f24d 0x551ee0 0x54efc1 0x54f24d 0x551ee0 0x54e4c8 0x54f4f6 0x553aaf 0x54e4c8
0.4.0
True


In [0]:
import torch 
import torch.nn as nn

In [0]:
# Logistic Regression for MNIST
# Hyper-parameters 
input_size = 239
num_classes = 1
num_epochs = 5
batch_size = 1000
learning_rate = 0.001

In [0]:
train_torch = torch.from_numpy(train)
train_torch = train_torch.type(torch.float)


val_torch = torch.from_numpy(val)
val_torch = val_torch.type(torch.float)

In [0]:
train_torch_label = train_label.astype('float32')
val_torch_label = val_label.astype('float32')


In [0]:
train_torch_label = torch.from_numpy(np.array(train_label.astype('int32')))
train_torch_label = train_torch_label.type(torch.LongTensor)

In [82]:
shape_check = np.array(train_label.values.reshape(-1,1))
print(shape_check.shape)
print(shape_check.dtype)

(246009, 1)
int64


In [0]:
train_torch_label = torch.from_numpy(shape_check)
train_torch_label = train_torch_label.type(torch.FloatTensor)

In [78]:
print(train_torch_label.shape)
print(train_torch.shape)

torch.Size([246009])
torch.Size([246009, 239])


In [88]:
model = nn.Linear(input_size,1) # We try to predict the value on output (eg: House price ) with four given feature (room size, etc..)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=.001) # linear parameters are w, b which we get using linear.weight , linear.bias
#print ('Weight & Bias Before Training ',model.weight,'&', model.bias)
for epoch in range(100):
  # Forward_pass
  pred = model(train_torch) # give the data to calculate forward linear layer

  loss = criterion(pred,train_torch_label) # compare prediction vs actuals
  if (epoch + 1) % 10 == 0 :
    print ('Epoch : loss ' , epoch, loss.item())

  # Make gradient to be zero
  optimizer.zero_grad()

  # Backward pass  Auto differntiation not much of an worry
  loss.backward()

  #Update using optimizer after calculating loss
  optimizer.step()

Epoch : loss  0 0.6775982975959778
